# Description

(Please, take a look at the README.md file in this directory for instructions on how to run this notebook)

This notebook compiles information about the GWAS and TWAS for a particular cohort. For example, the set of GWAS variants, variance of predicted expression of genes, etc.

It has specicfic parameters for papermill (see under `Settings` below).

This notebook is not directly run. See README.md.

# Modules

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
import pickle

import numpy as np
import pandas as pd

import conf
from entity import Gene

# Settings

In [3]:
# a cohort name (it could be something like UK_BIOBANK, etc)
COHORT_NAME = None

# reference panel such as 1000G or GTEX_V8
REFERENCE_PANEL = None

# predictions models such as MASHR or ELASTIC_NET
EQTL_MODEL = None

# a string with a path pointing to an imputed GWAS
GWAS_FILE = None

# a string with a path pointing where S-PrediXcan results (tissue-specific are located
SPREDIXCAN_FOLDER = None

# an f-string with one placeholder {tissue}
SPREDIXCAN_FILE_PATTERN = None

# a string with a path pointing to an S-MultiXcan result
SMULTIXCAN_FILE = None

In [4]:
# Parameters
COHORT_NAME = "phenomexcan_other"
REFERENCE_PANEL = "GTEX_V8"
EQTL_MODEL = "MASHR"
GWAS_FILE = "/project/ritchie20/projects/phenoplier/base/data/phenomexcan/gwas_parsing/full/MAGNETIC_IDL.TG.txt.gz"
SPREDIXCAN_FOLDER = "/project/ritchie20/projects/phenoplier/base/data/phenomexcan/spredixcan/gtex_gwas/MAGNETIC_IDL.TG/"
SPREDIXCAN_FILE_PATTERN = (
    "spredixcan_igwas_gtexmashrv8_MAGNETIC_IDL.TG__PM__{tissue}.csv"
)
SMULTIXCAN_FILE = "/project/ritchie20/projects/phenoplier/base/data/phenomexcan/smultixcan/gtex_gwas/MAGNETIC_IDL.TG_smultixcan_imputed_gwas_gtexv8mashr_ccn30.txt.gz"


In [5]:
assert COHORT_NAME is not None and len(COHORT_NAME) > 0, "A cohort name must be given"

COHORT_NAME = COHORT_NAME.lower()
display(f"Cohort name: {COHORT_NAME}")

'Cohort name: phenomexcan_other'

In [6]:
assert (
    REFERENCE_PANEL is not None and len(REFERENCE_PANEL) > 0
), "A reference panel must be given"

display(f"Reference panel: {REFERENCE_PANEL}")

'Reference panel: GTEX_V8'

In [7]:
assert GWAS_FILE is not None and len(GWAS_FILE) > 0, "A GWAS file path must be given"
GWAS_FILE = Path(GWAS_FILE).resolve()
assert GWAS_FILE.exists(), "GWAS file does not exist"

display(f"GWAS file path: {str(GWAS_FILE)}")

'GWAS file path: /project/ritchie20/projects/phenoplier/base/data/phenomexcan/gwas_parsing/full/MAGNETIC_IDL.TG.txt.gz'

In [8]:
assert (
    SPREDIXCAN_FOLDER is not None and len(SPREDIXCAN_FOLDER) > 0
), "An S-PrediXcan folder path must be given"
SPREDIXCAN_FOLDER = Path(SPREDIXCAN_FOLDER).resolve()
assert SPREDIXCAN_FOLDER.exists(), "S-PrediXcan folder does not exist"

display(f"S-PrediXcan folder path: {str(SPREDIXCAN_FOLDER)}")

'S-PrediXcan folder path: /project/ritchie20/projects/phenoplier/base/data/phenomexcan/spredixcan/gtex_gwas/MAGNETIC_IDL.TG'

In [9]:
assert (
    SPREDIXCAN_FILE_PATTERN is not None and len(SPREDIXCAN_FILE_PATTERN) > 0
), "An S-PrediXcan file pattern must be given"
assert (
    "{tissue}" in SPREDIXCAN_FILE_PATTERN
), "S-PrediXcan file pattern must have a '{tissue}' placeholder"

display(f"S-PrediXcan file template: {SPREDIXCAN_FILE_PATTERN}")

'S-PrediXcan file template: spredixcan_igwas_gtexmashrv8_MAGNETIC_IDL.TG__PM__{tissue}.csv'

In [10]:
assert (
    SMULTIXCAN_FILE is not None and len(SMULTIXCAN_FILE) > 0
), "An S-MultiXcan result file path must be given"
SMULTIXCAN_FILE = Path(SMULTIXCAN_FILE).resolve()
assert SMULTIXCAN_FILE.exists(), "S-MultiXcan result file does not exist"

display(f"S-MultiXcan file path: {str(SMULTIXCAN_FILE)}")

'S-MultiXcan file path: /project/ritchie20/projects/phenoplier/base/data/phenomexcan/smultixcan/gtex_gwas/MAGNETIC_IDL.TG_smultixcan_imputed_gwas_gtexv8mashr_ccn30.txt.gz'

In [11]:
assert (
    EQTL_MODEL is not None and len(EQTL_MODEL) > 0
), "A prediction/eQTL model must be given"

display(f"eQTL model: {EQTL_MODEL}")

'eQTL model: MASHR'

In [12]:
OUTPUT_DIR_BASE = (
    conf.RESULTS["GLS"]
    / "gene_corrs"
    / "cohorts"
    / COHORT_NAME
    / REFERENCE_PANEL.lower()
    / EQTL_MODEL.lower()
)

OUTPUT_DIR_BASE.mkdir(parents=True, exist_ok=True)

display(f"Using output dir base: {OUTPUT_DIR_BASE}")

'Using output dir base: /project/ritchie20/projects/phenoplier/base/results/gls/gene_corrs/cohorts/phenomexcan_other/gtex_v8/mashr'

# Load MultiPLIER Z genes

In [13]:
multiplier_z_genes = pd.read_pickle(
    conf.MULTIPLIER["MODEL_Z_MATRIX_FILE"]
).index.tolist()

In [14]:
len(multiplier_z_genes)

6750

In [15]:
assert len(multiplier_z_genes) == len(set(multiplier_z_genes))

In [16]:
multiplier_z_genes[:5]

['GAS6', 'MMP14', 'DSP', 'MARCKSL1', 'SPARC']

# GWAS

In [17]:
gwas_file_columns = pd.read_csv(GWAS_FILE, sep="\t", nrows=2).columns
assert (
    "panel_variant_id" in gwas_file_columns
), "GWAS file must be final imputed one with column 'panel_variant_id'"
# FIXME: add other needed columns here

In [18]:
gwas_data = pd.read_csv(
    GWAS_FILE,
    sep="\t",
    usecols=["panel_variant_id", "pvalue", "zscore"],
)

In [19]:
gwas_data.shape

(8865800, 3)

In [20]:
gwas_data.head()

,panel_variant_id,zscore,pvalue
0,chr1_13550_G_A_b38,0.165489,0.868559
1,chr1_14671_G_C_b38,0.116730,0.907074
2,chr1_14677_G_A_b38,1.021063,0.307225
3,chr1_14933_G_A_b38,-0.743508,0.457174
4,chr1_16841_G_T_b38,0.734379,0.462718


In [21]:
gwas_data.dropna().shape

(8865800, 3)

In [22]:
# remove SNPs with no results
gwas_data = gwas_data.dropna()

In [23]:
gwas_data.shape

(8865800, 3)

## Save GWAS variants

In [24]:
gwas_data.head()

,panel_variant_id,zscore,pvalue
0,chr1_13550_G_A_b38,0.165489,0.868559
1,chr1_14671_G_C_b38,0.116730,0.907074
2,chr1_14677_G_A_b38,1.021063,0.307225
3,chr1_14933_G_A_b38,-0.743508,0.457174
4,chr1_16841_G_T_b38,0.734379,0.462718


In [25]:
assert gwas_data["panel_variant_id"].is_unique

In [26]:
gwas_variants_ids_set = frozenset(gwas_data["panel_variant_id"])
list(gwas_variants_ids_set)[:5]

['chr8_72128927_C_A_b38',
 'chr8_5194038_T_C_b38',
 'chr12_41618615_T_TG_b38',
 'chr8_22423950_C_T_b38',
 'chr1_175935173_A_T_b38']

In [27]:
with open(OUTPUT_DIR_BASE / "gwas_variant_ids.pkl", "wb") as handle:
    pickle.dump(gwas_variants_ids_set, handle, protocol=pickle.HIGHEST_PROTOCOL)

# TWAS

## Available tissues for eQTL model

In [28]:
prediction_model_tissues = conf.PHENOMEXCAN["PREDICTION_MODELS"][
    f"{EQTL_MODEL}_TISSUES"
].split(" ")

In [29]:
len(prediction_model_tissues)

49

In [30]:
prediction_model_tissues[:5]

['Whole_Blood',
 'Brain_Hypothalamus',
 'Brain_Amygdala',
 'Brain_Nucleus_accumbens_basal_ganglia',
 'Artery_Coronary']

## S-MultiXcan results

In [31]:
smultixcan_results = pd.read_csv(
    SMULTIXCAN_FILE, sep="\t", usecols=["gene", "gene_name", "pvalue", "n", "n_indep"]
)

In [32]:
smultixcan_results.shape

(22515, 5)

In [33]:
smultixcan_results = smultixcan_results.dropna()

In [34]:
smultixcan_results.shape

(22512, 5)

In [35]:
smultixcan_results = smultixcan_results.assign(
    gene_id=smultixcan_results["gene"].apply(lambda g: g.split(".")[0])
)

In [36]:
smultixcan_results.head()

,gene,gene_name,pvalue,n,n_indep,gene_id
0,ENSG00000130203.9,APOE,4.399372e-77,42.0,9.0,ENSG00000130203
1,ENSG00000130202.9,NECTIN2,3.594063e-53,46.0,12.0,ENSG00000130202
2,ENSG00000130164.13,LDLR,2.672258e-39,13.0,3.0,ENSG00000130164
3,ENSG00000169174.10,PCSK9,9.023441e-31,33.0,11.0,ENSG00000169174
4,ENSG00000130204.12,TOMM40,6.407789e-30,45.0,4.0,ENSG00000130204


In [37]:
assert smultixcan_results["gene_id"].is_unique

### Get common genes with MultiPLIER

In [38]:
common_genes = set(multiplier_z_genes).intersection(
    set(smultixcan_results["gene_name"])
)

In [39]:
len(common_genes)

6452

In [40]:
sorted(list(common_genes))[:5]

['A2M', 'AAAS', 'AANAT', 'AARS', 'AARS2']

## Genes info

In [41]:
multiplier_gene_obj = {
    gene_name: Gene(name=gene_name)
    for gene_name in common_genes
    if gene_name in Gene.GENE_NAME_TO_ID_MAP
}

In [42]:
# delete common_genes, from now on, genes_info should be used for common genes
del common_genes

In [43]:
len(multiplier_gene_obj)

6452

In [44]:
assert multiplier_gene_obj["GAS6"].ensembl_id == "ENSG00000183087"

In [45]:
_gene_obj = list(multiplier_gene_obj.values())

genes_info = pd.DataFrame(
    {
        "name": [g.name for g in _gene_obj],
        "id": [g.ensembl_id for g in _gene_obj],
        "chr": [g.chromosome for g in _gene_obj],
        "band": [g.band for g in _gene_obj],
        "start_position": [g.get_attribute("start_position") for g in _gene_obj],
        "end_position": [g.get_attribute("end_position") for g in _gene_obj],
    }
)

In [46]:
genes_info = genes_info.assign(
    gene_length=genes_info.apply(
        lambda x: x["end_position"] - x["start_position"], axis=1
    )
)

In [47]:
genes_info.dtypes

name               object
id                 object
chr                object
band               object
start_position    float64
end_position      float64
gene_length       float64
dtype: object

In [48]:
_tmp = genes_info[genes_info.isna().any(axis=1)]
display(_tmp)
assert _tmp.shape[0] < 5

,name,id,chr,band,start_position,end_position,gene_length
4903,TBCE,ENSG00000116957,None,None,NaN,NaN,NaN
5403,TMEM133,ENSG00000170647,None,None,NaN,NaN,NaN


In [49]:
genes_info = genes_info.dropna()

In [50]:
genes_info["chr"] = genes_info["chr"].apply(pd.to_numeric, downcast="integer")
genes_info["start_position"] = genes_info["start_position"].astype(int)
genes_info["end_position"] = genes_info["end_position"].astype(int)
genes_info["gene_length"] = genes_info["gene_length"].astype(int)

In [51]:
genes_info.dtypes

name              object
id                object
chr                int64
band              object
start_position     int64
end_position       int64
gene_length        int64
dtype: object

In [52]:
assert genes_info["name"].is_unique

In [53]:
assert genes_info["id"].is_unique

In [54]:
genes_info.shape

(6450, 7)

In [55]:
genes_info.head()

,name,id,chr,band,start_position,end_position,gene_length
0,FUS,ENSG00000089280,16,16p11.2,31180110,31194871,14761
1,PI4K2A,ENSG00000155252,10,10q24.2,97640686,97676434,35748
2,PER1,ENSG00000179094,17,17p13.1,8140472,8156506,16034
3,GBP6,ENSG00000183347,1,1p22.2,89364058,89386461,22403
4,GPAA1,ENSG00000197858,8,8q24.3,144082590,144086216,3626


In [56]:
genes_info.sort_values("chr")

,name,id,chr,band,start_position,end_position,gene_length
649,SPSB1,ENSG00000171621,1,1p36.22,9292894,9369532,76638
6223,SV2A,ENSG00000159164,1,1q21.2,149903318,149917844,14526
6222,MTMR11,ENSG00000014914,1,1q21.2,149928651,149936879,8228
6221,KCNJ10,ENSG00000177807,1,1q23.2,159998651,160070483,71832
737,OPN3,ENSG00000054277,1,1q43,241590102,241677376,87274
...,...,...,...,...,...,...,...
1660,AP1B1,ENSG00000100280,22,22q12.2,29327680,29423179,95499
1310,UPK3A,ENSG00000100373,22,22q13.31,45284949,45295874,10925
4177,POLR2F,ENSG00000100142,22,22q13.1,37952607,38041915,89308
1669,CRYBB1,ENSG00000100122,22,22q12.1,26599278,26618027,18749


### Save

In [57]:
genes_info.to_pickle(OUTPUT_DIR_BASE / "genes_info.pkl")

## S-PrediXcan results

### Load results across all tissues

In [58]:
spredixcan_result_files = {
    t: SPREDIXCAN_FOLDER / SPREDIXCAN_FILE_PATTERN.format(tissue=t)
    for t in prediction_model_tissues
}

In [59]:
assert len(spredixcan_result_files) == len(prediction_model_tissues)
display(list(spredixcan_result_files.values())[:5])

[PosixPath('/project/ritchie20/projects/phenoplier/base/data/phenomexcan/spredixcan/gtex_gwas/MAGNETIC_IDL.TG/spredixcan_igwas_gtexmashrv8_MAGNETIC_IDL.TG__PM__Whole_Blood.csv'),
 PosixPath('/project/ritchie20/projects/phenoplier/base/data/phenomexcan/spredixcan/gtex_gwas/MAGNETIC_IDL.TG/spredixcan_igwas_gtexmashrv8_MAGNETIC_IDL.TG__PM__Brain_Hypothalamus.csv'),
 PosixPath('/project/ritchie20/projects/phenoplier/base/data/phenomexcan/spredixcan/gtex_gwas/MAGNETIC_IDL.TG/spredixcan_igwas_gtexmashrv8_MAGNETIC_IDL.TG__PM__Brain_Amygdala.csv'),
 PosixPath('/project/ritchie20/projects/phenoplier/base/data/phenomexcan/spredixcan/gtex_gwas/MAGNETIC_IDL.TG/spredixcan_igwas_gtexmashrv8_MAGNETIC_IDL.TG__PM__Brain_Nucleus_accumbens_basal_ganglia.csv'),
 PosixPath('/project/ritchie20/projects/phenoplier/base/data/phenomexcan/spredixcan/gtex_gwas/MAGNETIC_IDL.TG/spredixcan_igwas_gtexmashrv8_MAGNETIC_IDL.TG__PM__Artery_Coronary.csv')]

In [60]:
# look at the structure of one result
pd.read_csv(spredixcan_result_files["Whole_Blood"]).head()

,gene,gene_name,zscore,effect_size,pvalue,var_g,pred_perf_r2,pred_perf_pval,pred_perf_qval,n_snps_used,n_snps_in_cov,n_snps_in_model,best_gwas_p,largest_weight
0,ENSG00000130203.9,APOE,17.582577,NaN,3.349999e-69,0.004660,NaN,NaN,NaN,1,1.0,1,3.349999e-69,0.151524
1,ENSG00000234906.9,APOC2,11.715613,NaN,1.060222e-31,0.002668,NaN,NaN,NaN,2,2.0,2,1.410008e-32,0.249415
2,ENSG00000134222.16,PSRC1,-9.399907,NaN,5.461119e-21,0.060638,NaN,NaN,NaN,2,2.0,2,1.100000e-23,0.277846
3,ENSG00000162399.6,BSND,9.240837,NaN,2.445770e-20,0.113038,NaN,NaN,NaN,3,3.0,3,1.710004e-16,0.353909
4,ENSG00000169174.10,PCSK9,9.204915,NaN,3.419418e-20,0.045947,NaN,NaN,NaN,4,4.0,4,5.109546e-17,0.204700


In [61]:
assert all(f.exists() for f in spredixcan_result_files.values())

In [62]:
spredixcan_dfs = [
    pd.read_csv(
        f,
        usecols=[
            "gene",
            "zscore",
            "pvalue",
            "n_snps_used",
            "n_snps_in_model",
        ],
    )
    .dropna(subset=["gene", "zscore", "pvalue"])
    .assign(tissue=t)
    for t, f in spredixcan_result_files.items()
]

In [63]:
assert len(spredixcan_dfs) == len(prediction_model_tissues)

In [64]:
spredixcan_dfs = pd.concat(spredixcan_dfs)

In [65]:
assert spredixcan_dfs["tissue"].unique().shape[0] == len(prediction_model_tissues)

In [66]:
spredixcan_dfs.shape

(684160, 6)

In [67]:
spredixcan_dfs = spredixcan_dfs.assign(
    gene_id=spredixcan_dfs["gene"].apply(lambda g: g.split(".")[0])
)

In [68]:
spredixcan_dfs.head()

,gene,zscore,pvalue,n_snps_used,n_snps_in_model,tissue,gene_id
0,ENSG00000130203.9,17.582577,3.349999e-69,1,1,Whole_Blood,ENSG00000130203
1,ENSG00000234906.9,11.715613,1.060222e-31,2,2,Whole_Blood,ENSG00000234906
2,ENSG00000134222.16,-9.399907,5.461119e-21,2,2,Whole_Blood,ENSG00000134222
3,ENSG00000162399.6,9.240837,2.445770e-20,3,3,Whole_Blood,ENSG00000162399
4,ENSG00000169174.10,9.204915,3.419418e-20,4,4,Whole_Blood,ENSG00000169174


In [69]:
# leave only common genes
spredixcan_dfs = spredixcan_dfs[spredixcan_dfs["gene_id"].isin(set(genes_info["id"]))]

In [70]:
spredixcan_dfs.shape

(243879, 7)

### Count number of tissues available per gene

In [71]:
spredixcan_genes_n_models = spredixcan_dfs.groupby("gene_id")["tissue"].nunique()

In [72]:
spredixcan_genes_n_models

gene_id
ENSG00000000419     3
ENSG00000000938    36
ENSG00000000971    34
ENSG00000001084    33
ENSG00000001167    40
                   ..
ENSG00000278540    36
ENSG00000278828     4
ENSG00000278845    49
ENSG00000281005    49
ENSG00000282608    36
Name: tissue, Length: 6450, dtype: int64

In [73]:
# testing that in S-MultiXcan I get the same number of tissues per gene
_tmp_smultixcan_results_n_models = (
    smultixcan_results.set_index("gene_id")["n"].astype(int).rename("tissue")
)

_cg = _tmp_smultixcan_results_n_models.index.intersection(
    spredixcan_genes_n_models.index
)
_tmp_smultixcan_results_n_models = _tmp_smultixcan_results_n_models.loc[_cg]
_spredixcan = spredixcan_genes_n_models.loc[_cg]

assert _spredixcan.shape[0] == _tmp_smultixcan_results_n_models.shape[0]
assert _spredixcan.equals(_tmp_smultixcan_results_n_models.loc[_spredixcan.index])

### Get tissues available per gene

In [74]:
spredixcan_genes_models = spredixcan_dfs.groupby("gene_id")["tissue"].apply(
    lambda x: frozenset(x.tolist())
)

In [75]:
spredixcan_genes_models

gene_id
ENSG00000000419    (Brain_Hypothalamus, Brain_Substantia_nigra, C...
ENSG00000000938    (Brain_Cortex, Artery_Tibial, Testis, Adipose_...
ENSG00000000971    (Brain_Cortex, Artery_Tibial, Cells_Cultured_f...
ENSG00000001084    (Brain_Cortex, Artery_Tibial, Cells_Cultured_f...
ENSG00000001167    (Brain_Cortex, Artery_Tibial, Cells_Cultured_f...
                                         ...                        
ENSG00000278540    (Brain_Cortex, Artery_Tibial, Cells_Cultured_f...
ENSG00000278828    (Cells_EBV-transformed_lymphocytes, Esophagus_...
ENSG00000278845    (Brain_Cortex, Artery_Tibial, Cells_Cultured_f...
ENSG00000281005    (Brain_Cortex, Artery_Tibial, Cells_Cultured_f...
ENSG00000282608    (Brain_Cortex, Artery_Tibial, Testis, Adipose_...
Name: tissue, Length: 6450, dtype: object

In [76]:
assert spredixcan_genes_n_models.shape[0] == spredixcan_genes_models.shape[0]

In [77]:
assert spredixcan_genes_n_models.index.equals(spredixcan_genes_models.index)

In [78]:
assert (spredixcan_genes_models.apply(len) <= len(prediction_model_tissues)).all()

In [79]:
spredixcan_genes_models.apply(len).describe()

count    6450.000000
mean       37.810698
std        12.468797
min         1.000000
25%        32.000000
50%        43.000000
75%        47.000000
max        49.000000
Name: tissue, dtype: float64

In [80]:
# testing
assert (
    spredixcan_genes_models.loc[spredixcan_genes_n_models.index]
    .apply(len)
    .equals(spredixcan_genes_n_models)
)

### Add gene name and set index

In [81]:
spredixcan_genes_models = spredixcan_genes_models.to_frame().reset_index()

In [82]:
spredixcan_genes_models.head()

,gene_id,tissue
0,ENSG00000000419,"(Brain_Hypothalamus, Brain_Substantia_nigra, C..."
1,ENSG00000000938,"(Brain_Cortex, Artery_Tibial, Testis, Adipose_..."
2,ENSG00000000971,"(Brain_Cortex, Artery_Tibial, Cells_Cultured_f..."
3,ENSG00000001084,"(Brain_Cortex, Artery_Tibial, Cells_Cultured_f..."
4,ENSG00000001167,"(Brain_Cortex, Artery_Tibial, Cells_Cultured_f..."


In [83]:
spredixcan_genes_models = spredixcan_genes_models.assign(
    gene_name=spredixcan_genes_models["gene_id"].apply(
        lambda g: Gene.GENE_ID_TO_NAME_MAP[g]
    )
)

In [84]:
spredixcan_genes_models = spredixcan_genes_models[["gene_id", "gene_name", "tissue"]]

In [85]:
spredixcan_genes_models = spredixcan_genes_models.set_index("gene_id")

In [86]:
spredixcan_genes_models.head()

,gene_name,tissue
gene_id,,
ENSG00000000419,DPM1,"(Brain_Hypothalamus, Brain_Substantia_nigra, C..."
ENSG00000000938,FGR,"(Brain_Cortex, Artery_Tibial, Testis, Adipose_..."
ENSG00000000971,CFH,"(Brain_Cortex, Artery_Tibial, Cells_Cultured_f..."
ENSG00000001084,GCLC,"(Brain_Cortex, Artery_Tibial, Cells_Cultured_f..."
ENSG00000001167,NFYA,"(Brain_Cortex, Artery_Tibial, Cells_Cultured_f..."


### Add number of tissues

In [87]:
spredixcan_genes_models = spredixcan_genes_models.assign(
    n_tissues=spredixcan_genes_models["tissue"].apply(len)
)

In [88]:
spredixcan_genes_models.head()

,gene_name,tissue,n_tissues
gene_id,,,
ENSG00000000419,DPM1,"(Brain_Hypothalamus, Brain_Substantia_nigra, C...",3
ENSG00000000938,FGR,"(Brain_Cortex, Artery_Tibial, Testis, Adipose_...",36
ENSG00000000971,CFH,"(Brain_Cortex, Artery_Tibial, Cells_Cultured_f...",34
ENSG00000001084,GCLC,"(Brain_Cortex, Artery_Tibial, Cells_Cultured_f...",33
ENSG00000001167,NFYA,"(Brain_Cortex, Artery_Tibial, Cells_Cultured_f...",40


### Save

Here I quickly save these results to a file, given that the next steps (covariates) are slow to compute.

In [89]:
# this is important, other scripts depend on gene_name to be unique
assert spredixcan_genes_models["gene_name"].is_unique

In [90]:
assert not spredixcan_genes_models.isna().any(None)

In [91]:
spredixcan_genes_models.to_pickle(OUTPUT_DIR_BASE / "gene_tissues.pkl")

## Add covariates based on S-PrediXcan results

This extend the previous file with more columns

### Get gene's objects

In [92]:
spredixcan_gene_obj = {
    gene_id: Gene(ensembl_id=gene_id) for gene_id in spredixcan_genes_models.index
}

In [93]:
len(spredixcan_gene_obj)

6450

### Add genes' variance captured by principal components

In [94]:
def _get_gene_pc_variance(gene_row):
    gene_id = gene_row.name
    gene_tissues = gene_row["tissue"]
    gene_obj = spredixcan_gene_obj[gene_id]

    u, s, vt = gene_obj.get_tissues_correlations_svd(
        tissues=gene_tissues,
        snps_subset=gwas_variants_ids_set,
        reference_panel=REFERENCE_PANEL,
        model_type=EQTL_MODEL,
        # use_covariance_matrix=True,
    )

    return s

In [95]:
_tmp = spredixcan_genes_models.loc["ENSG00000188976"]
_get_gene_pc_variance(_tmp)

array([35.60509556,  3.87543604,  2.16044422,  1.41758395,  1.26548868])

In [96]:
spredixcan_genes_tissues_pc_variance = spredixcan_genes_models.apply(
    _get_gene_pc_variance, axis=1
)

In [97]:
spredixcan_genes_tissues_pc_variance

gene_id
ENSG00000000419    [1.0525406583994028, 1.024362410165966, 0.9230...
ENSG00000000938    [31.63266933637127, 2.0783677518511445, 1.2716...
ENSG00000000971    [21.560769925683296, 7.3101709870454625, 1.837...
ENSG00000001084    [21.74736589643256, 4.70064578668365, 2.249949...
ENSG00000001167                                  [38.28994327673914]
                                         ...                        
ENSG00000278540    [30.239540272954727, 3.0448326465798914, 1.679...
ENSG00000278828              [3.062182335109642, 0.9378176648903574]
ENSG00000278845               [45.6143801458816, 2.2705553628003416]
ENSG00000281005                                  [48.30170739785572]
ENSG00000282608    [22.653514081237113, 6.245520030604539, 2.5559...
Length: 6450, dtype: object

In [98]:
# # testing
# assert spredixcan_genes_tissues_pc_variance.loc[
#     "ENSG00000188976"
# ].sum() == pytest.approx(44.01605629086847)
# # this is using the covariance:
# # assert spredixcan_genes_tissues_pc_variance.loc["ENSG00000188976"].sum() == pytest.approx(1.1492946006449425)

In [99]:
# add to spredixcan_genes_models
spredixcan_genes_models = spredixcan_genes_models.join(
    spredixcan_genes_tissues_pc_variance.rename("tissues_pc_variances")
)

In [100]:
spredixcan_genes_models.shape

(6450, 4)

In [101]:
spredixcan_genes_models.head()

,gene_name,tissue,n_tissues,tissues_pc_variances
gene_id,,,,
ENSG00000000419,DPM1,"(Brain_Hypothalamus, Brain_Substantia_nigra, C...",3,"[1.0525406583994028, 1.024362410165966, 0.9230..."
ENSG00000000938,FGR,"(Brain_Cortex, Artery_Tibial, Testis, Adipose_...",36,"[31.63266933637127, 2.0783677518511445, 1.2716..."
ENSG00000000971,CFH,"(Brain_Cortex, Artery_Tibial, Cells_Cultured_f...",34,"[21.560769925683296, 7.3101709870454625, 1.837..."
ENSG00000001084,GCLC,"(Brain_Cortex, Artery_Tibial, Cells_Cultured_f...",33,"[21.74736589643256, 4.70064578668365, 2.249949..."
ENSG00000001167,NFYA,"(Brain_Cortex, Artery_Tibial, Cells_Cultured_f...",40,[38.28994327673914]


### Add gene variance per tissue

In [102]:
def _get_gene_variances(gene_row):
    gene_id = gene_row.name
    gene_tissues = gene_row["tissue"]

    tissue_variances = {}
    gene_obj = spredixcan_gene_obj[gene_id]

    for tissue in gene_tissues:
        tissue_var = gene_obj.get_pred_expression_variance(
            tissue=tissue,
            reference_panel=REFERENCE_PANEL,
            model_type=EQTL_MODEL,
            snps_subset=gwas_variants_ids_set,
        )

        if tissue_var is not None:
            tissue_variances[tissue] = tissue_var

    return tissue_variances

In [103]:
_tmp = spredixcan_genes_models.loc["ENSG00000000419"]
_get_gene_variances(_tmp)

{'Brain_Hypothalamus': 0.013162153504206677,
 'Brain_Substantia_nigra': 0.0004867380334260178,
 'Cells_Cultured_fibroblasts': 0.0030618005265686445}

In [104]:
spredixcan_genes_tissues_variance = spredixcan_genes_models.apply(
    _get_gene_variances, axis=1
)

In [105]:
spredixcan_genes_tissues_variance

gene_id
ENSG00000000419    {'Brain_Hypothalamus': 0.013162153504206677, '...
ENSG00000000938    {'Brain_Cortex': 0.004882241504747395, 'Artery...
ENSG00000000971    {'Brain_Cortex': 0.005725070121841921, 'Artery...
ENSG00000001084    {'Brain_Cortex': 0.024826936262456585, 'Artery...
ENSG00000001167    {'Brain_Cortex': 0.016199723974989217, 'Artery...
                                         ...                        
ENSG00000278540    {'Brain_Cortex': 0.0056851674728940725, 'Arter...
ENSG00000278828    {'Cells_EBV-transformed_lymphocytes': 0.003027...
ENSG00000278845    {'Brain_Cortex': 0.07259083788627797, 'Artery_...
ENSG00000281005    {'Brain_Cortex': 0.1386141871539704, 'Artery_T...
ENSG00000282608    {'Brain_Cortex': 0.022567360072040715, 'Artery...
Length: 6450, dtype: object

In [106]:
# # testing
# _gene_id = "ENSG00000188976"
# x = spredixcan_genes_tissues_variance.loc[_gene_id]
# # expected value obtained by sum of PCA eigenvalues on this gene's predicted expression
# assert np.sum(list(x.values())) == pytest.approx(1.2326202607409493)

In [107]:
# testing
spredixcan_genes_tissues_variance.loc["ENSG00000000419"]

{'Brain_Hypothalamus': 0.013162153504206677,
 'Brain_Substantia_nigra': 0.0004867380334260178,
 'Cells_Cultured_fibroblasts': 0.0030618005265686445}

In [108]:
# FIXME: maybe add more tests, these differt from GTEX V8
# # testing
# # here values were obtained from S-PrediXcan results, where the reference panel is GTEX V8, not 1000G, so just approximations)
# _gene_id = "ENSG00000000419"
# assert spredixcan_genes_tissues_variance.loc[_gene_id]["Brain_Substantia_nigra"] == pytest.approx(0.0004266255268163448)
# assert spredixcan_genes_tissues_variance.loc[_gene_id]["Brain_Hypothalamus"] == pytest.approx(0.011235877515236132)

In [109]:
# add to spredixcan_genes_models
spredixcan_genes_models = spredixcan_genes_models.join(
    spredixcan_genes_tissues_variance.rename("tissues_variances")
)

In [110]:
spredixcan_genes_models.shape

(6450, 5)

In [111]:
spredixcan_genes_models.head()

,gene_name,tissue,n_tissues,tissues_pc_variances,tissues_variances
gene_id,,,,,
ENSG00000000419,DPM1,"(Brain_Hypothalamus, Brain_Substantia_nigra, C...",3,"[1.0525406583994028, 1.024362410165966, 0.9230...","{'Brain_Hypothalamus': 0.013162153504206677, '..."
ENSG00000000938,FGR,"(Brain_Cortex, Artery_Tibial, Testis, Adipose_...",36,"[31.63266933637127, 2.0783677518511445, 1.2716...","{'Brain_Cortex': 0.004882241504747395, 'Artery..."
ENSG00000000971,CFH,"(Brain_Cortex, Artery_Tibial, Cells_Cultured_f...",34,"[21.560769925683296, 7.3101709870454625, 1.837...","{'Brain_Cortex': 0.005725070121841921, 'Artery..."
ENSG00000001084,GCLC,"(Brain_Cortex, Artery_Tibial, Cells_Cultured_f...",33,"[21.74736589643256, 4.70064578668365, 2.249949...","{'Brain_Cortex': 0.024826936262456585, 'Artery..."
ENSG00000001167,NFYA,"(Brain_Cortex, Artery_Tibial, Cells_Cultured_f...",40,[38.28994327673914],"{'Brain_Cortex': 0.016199723974989217, 'Artery..."


### Count number of SNPs predictors used across tissue models

In [112]:
spredixcan_genes_sum_of_n_snps_used = (
    spredixcan_dfs.groupby("gene_id")["n_snps_used"].sum().rename("n_snps_used_sum")
)

In [113]:
spredixcan_genes_sum_of_n_snps_used

gene_id
ENSG00000000419     3
ENSG00000000938    40
ENSG00000000971    44
ENSG00000001084    47
ENSG00000001167    48
                   ..
ENSG00000278540    44
ENSG00000278828     5
ENSG00000278845    91
ENSG00000281005    81
ENSG00000282608    40
Name: n_snps_used_sum, Length: 6450, dtype: int64

In [114]:
# add sum of snps used to spredixcan_genes_models
spredixcan_genes_models = spredixcan_genes_models.join(
    spredixcan_genes_sum_of_n_snps_used
)

In [115]:
spredixcan_genes_models.shape

(6450, 6)

In [116]:
spredixcan_genes_models.head()

,gene_name,tissue,n_tissues,tissues_pc_variances,tissues_variances,n_snps_used_sum
gene_id,,,,,,
ENSG00000000419,DPM1,"(Brain_Hypothalamus, Brain_Substantia_nigra, C...",3,"[1.0525406583994028, 1.024362410165966, 0.9230...","{'Brain_Hypothalamus': 0.013162153504206677, '...",3
ENSG00000000938,FGR,"(Brain_Cortex, Artery_Tibial, Testis, Adipose_...",36,"[31.63266933637127, 2.0783677518511445, 1.2716...","{'Brain_Cortex': 0.004882241504747395, 'Artery...",40
ENSG00000000971,CFH,"(Brain_Cortex, Artery_Tibial, Cells_Cultured_f...",34,"[21.560769925683296, 7.3101709870454625, 1.837...","{'Brain_Cortex': 0.005725070121841921, 'Artery...",44
ENSG00000001084,GCLC,"(Brain_Cortex, Artery_Tibial, Cells_Cultured_f...",33,"[21.74736589643256, 4.70064578668365, 2.249949...","{'Brain_Cortex': 0.024826936262456585, 'Artery...",47
ENSG00000001167,NFYA,"(Brain_Cortex, Artery_Tibial, Cells_Cultured_f...",40,[38.28994327673914],"{'Brain_Cortex': 0.016199723974989217, 'Artery...",48


### Count number of SNPs predictors in models across tissue models

In [117]:
spredixcan_genes_sum_of_n_snps_in_model = (
    spredixcan_dfs.groupby("gene_id")["n_snps_in_model"]
    .sum()
    .rename("n_snps_in_model_sum")
)

In [118]:
spredixcan_genes_sum_of_n_snps_in_model

gene_id
ENSG00000000419     3
ENSG00000000938    40
ENSG00000000971    44
ENSG00000001084    47
ENSG00000001167    48
                   ..
ENSG00000278540    44
ENSG00000278828     5
ENSG00000278845    91
ENSG00000281005    81
ENSG00000282608    40
Name: n_snps_in_model_sum, Length: 6450, dtype: int64

In [119]:
# add sum of snps in model to spredixcan_genes_models
spredixcan_genes_models = spredixcan_genes_models.join(
    spredixcan_genes_sum_of_n_snps_in_model
)

In [120]:
spredixcan_genes_models.shape

(6450, 7)

In [121]:
spredixcan_genes_models.head()

,gene_name,tissue,n_tissues,tissues_pc_variances,tissues_variances,n_snps_used_sum,n_snps_in_model_sum
gene_id,,,,,,,
ENSG00000000419,DPM1,"(Brain_Hypothalamus, Brain_Substantia_nigra, C...",3,"[1.0525406583994028, 1.024362410165966, 0.9230...","{'Brain_Hypothalamus': 0.013162153504206677, '...",3,3
ENSG00000000938,FGR,"(Brain_Cortex, Artery_Tibial, Testis, Adipose_...",36,"[31.63266933637127, 2.0783677518511445, 1.2716...","{'Brain_Cortex': 0.004882241504747395, 'Artery...",40,40
ENSG00000000971,CFH,"(Brain_Cortex, Artery_Tibial, Cells_Cultured_f...",34,"[21.560769925683296, 7.3101709870454625, 1.837...","{'Brain_Cortex': 0.005725070121841921, 'Artery...",44,44
ENSG00000001084,GCLC,"(Brain_Cortex, Artery_Tibial, Cells_Cultured_f...",33,"[21.74736589643256, 4.70064578668365, 2.249949...","{'Brain_Cortex': 0.024826936262456585, 'Artery...",47,47
ENSG00000001167,NFYA,"(Brain_Cortex, Artery_Tibial, Cells_Cultured_f...",40,[38.28994327673914],"{'Brain_Cortex': 0.016199723974989217, 'Artery...",48,48


### Summarize prediction models for each gene

In [122]:
def _summarize_gene_models(gene_id):
    """
    For a given gene ID, it returns a dataframe with predictor SNPs in rows and tissues in columns, where
    values are the weights of SNPs in those tissues.
    It can contain NaNs.
    """
    gene_obj = spredixcan_gene_obj[gene_id]
    gene_tissues = spredixcan_genes_models.loc[gene_id, "tissue"]

    gene_models = {}
    gene_unique_snps = set()
    for t in gene_tissues:
        gene_model = gene_obj.get_prediction_weights(tissue=t, model_type=EQTL_MODEL)
        gene_models[t] = gene_model

        gene_unique_snps.update(set(gene_model.index))

    df = pd.DataFrame(
        data=np.nan, index=list(gene_unique_snps), columns=list(gene_tissues)
    )

    for t in df.columns:
        for snp in df.index:
            gene_model = gene_models[t]

            if snp in gene_model.index:
                df.loc[snp, t] = gene_model.loc[snp]

    return df

In [123]:
# testing
spredixcan_gene_obj["ENSG00000000419"].get_prediction_weights(
    tissue="Brain_Hypothalamus", model_type=EQTL_MODEL
)

varID
chr20_50862947_C_T_b38    0.431375
Name: weight, dtype: float64

In [124]:
spredixcan_gene_obj["ENSG00000000419"].get_prediction_weights(
    tissue="Brain_Substantia_nigra", model_type=EQTL_MODEL
)

varID
chr20_50957480_C_T_b38   -0.146796
Name: weight, dtype: float64

In [125]:
# # testing
# _gene_id = "ENSG00000000419"

# _gene_model = _summarize_gene_models(_gene_id)
# assert (
#     _gene_model.loc["chr20_50862947_C_T_b38", "Brain_Hypothalamus"].round(5) == 0.43138
# )
# assert pd.isnull(_gene_model.loc["chr20_50957480_C_T_b38", "Brain_Hypothalamus"])

# assert pd.isnull(_gene_model.loc["chr20_50862947_C_T_b38", "Brain_Substantia_nigra"])
# assert (
#     _gene_model.loc["chr20_50957480_C_T_b38", "Brain_Substantia_nigra"].round(5)
#     == -0.1468
# )

In [126]:
gene_models = {}

for gene_id in spredixcan_genes_models.index:
    gene_models[gene_id] = _summarize_gene_models(gene_id)

In [127]:
# # testing
# _gene_id = "ENSG00000000419"

# _gene_model = gene_models[_gene_id]
# assert (
#     _gene_model.loc["chr20_50862947_C_T_b38", "Brain_Hypothalamus"].round(5) == 0.43138
# )
# assert pd.isnull(_gene_model.loc["chr20_50957480_C_T_b38", "Brain_Hypothalamus"])

# assert pd.isnull(_gene_model.loc["chr20_50862947_C_T_b38", "Brain_Substantia_nigra"])
# assert (
#     _gene_model.loc["chr20_50957480_C_T_b38", "Brain_Substantia_nigra"].round(5)
#     == -0.1468
# )

In [128]:
# save
import gzip

with gzip.GzipFile(OUTPUT_DIR_BASE / "gene_tissues_models.pkl.gz", "w") as f:
    pickle.dump(gene_models, f)

In [129]:
# testing saved file
with gzip.GzipFile(OUTPUT_DIR_BASE / "gene_tissues_models.pkl.gz", "r") as f:
    _tmp = pickle.load(f)

In [130]:
assert len(gene_models) == len(_tmp)
assert gene_models["ENSG00000000419"].equals(_tmp["ENSG00000000419"])

### Count number of _unique_ SNPs predictors used and available across tissue models

In [131]:
def _count_unique_snps(gene_id):
    """
    For a gene_id, it counts unique SNPs in all models and their intersection with GWAS SNPs (therefore, used by S-PrediXcan).
    """
    gene_tissues = spredixcan_genes_models.loc[gene_id, "tissue"]

    gene_unique_snps = set()
    for t in gene_tissues:
        t_snps = set(gene_models[gene_id].index)
        gene_unique_snps.update(t_snps)

    gene_unique_snps_in_gwas = gwas_variants_ids_set.intersection(gene_unique_snps)

    return pd.Series(
        {
            "unique_n_snps_in_model": len(gene_unique_snps),
            "unique_n_snps_used": len(gene_unique_snps_in_gwas),
        }
    )

In [132]:
# testing
spredixcan_genes_models[spredixcan_genes_models["n_snps_used_sum"] == 2].head()

,gene_name,tissue,n_tissues,tissues_pc_variances,tissues_variances,n_snps_used_sum,n_snps_in_model_sum
gene_id,,,,,,,
ENSG00000010256,UQCRC1,"(Thyroid, Whole_Blood)",2,"[1.1048350039574655, 0.8951649960425345]","{'Thyroid': 0.0008025245652862433, 'Whole_Bloo...",2,2
ENSG00000017427,IGF1,"(Brain_Amygdala, Testis)",2,"[1.0389319526404415, 0.9610680473595585]","{'Brain_Amygdala': 0.02059332044751194, 'Testi...",2,2
ENSG00000043093,DCUN1D1,"(Esophagus_Muscularis, Esophagus_Gastroesophag...",2,"[1.4951653523020276, 0.5048346476979725]",{'Esophagus_Muscularis': 0.0004315621222418269...,2,2
ENSG00000081377,CDC14B,"(Muscle_Skeletal, Brain_Nucleus_accumbens_basa...",2,"[1.1285007973441137, 0.8714992026558863]","{'Muscle_Skeletal': 0.0076846621297156385, 'Br...",2,2
ENSG00000104856,RELB,"(Brain_Cerebellar_Hemisphere, Esophagus_Gastro...",2,"[1.065634630043069, 0.9343653699569308]",{'Brain_Cerebellar_Hemisphere': 0.006361992573...,2,2


In [133]:
# case with two snps, not repeated across tissues
_gene_id = "ENSG00000000419"
display(
    spredixcan_gene_obj[_gene_id].get_prediction_weights(
        tissue="Brain_Hypothalamus", model_type=EQTL_MODEL
    )
)
display(
    spredixcan_gene_obj[_gene_id].get_prediction_weights(
        tissue="Brain_Substantia_nigra", model_type=EQTL_MODEL
    )
)

varID
chr20_50862947_C_T_b38    0.431375
Name: weight, dtype: float64

varID
chr20_50957480_C_T_b38   -0.146796
Name: weight, dtype: float64

In [134]:
# _tmp = _count_unique_snps(_gene_id)
# assert _tmp.shape[0] == 2
# assert _tmp["unique_n_snps_in_model"] == 2
# assert _tmp["unique_n_snps_used"] == 2

In [135]:
# get unique snps for all genes
spredixcan_genes_unique_n_snps = spredixcan_genes_models.groupby("gene_id").apply(
    lambda x: _count_unique_snps(x.name)
)

In [136]:
spredixcan_genes_unique_n_snps.head()

,unique_n_snps_in_model,unique_n_snps_used
gene_id,,
ENSG00000000419,3,3
ENSG00000000938,5,5
ENSG00000000971,12,12
ENSG00000001084,24,24
ENSG00000001167,14,14


In [137]:
assert (
    spredixcan_genes_unique_n_snps["unique_n_snps_in_model"]
    >= spredixcan_genes_unique_n_snps["unique_n_snps_used"]
).all()

In [138]:
# add unique snps to spredixcan_genes_models
spredixcan_genes_models = spredixcan_genes_models.join(spredixcan_genes_unique_n_snps)

In [139]:
spredixcan_genes_models.shape

(6450, 9)

In [140]:
spredixcan_genes_models.head()

,gene_name,tissue,n_tissues,tissues_pc_variances,tissues_variances,n_snps_used_sum,n_snps_in_model_sum,unique_n_snps_in_model,unique_n_snps_used
gene_id,,,,,,,,,
ENSG00000000419,DPM1,"(Brain_Hypothalamus, Brain_Substantia_nigra, C...",3,"[1.0525406583994028, 1.024362410165966, 0.9230...","{'Brain_Hypothalamus': 0.013162153504206677, '...",3,3,3,3
ENSG00000000938,FGR,"(Brain_Cortex, Artery_Tibial, Testis, Adipose_...",36,"[31.63266933637127, 2.0783677518511445, 1.2716...","{'Brain_Cortex': 0.004882241504747395, 'Artery...",40,40,5,5
ENSG00000000971,CFH,"(Brain_Cortex, Artery_Tibial, Cells_Cultured_f...",34,"[21.560769925683296, 7.3101709870454625, 1.837...","{'Brain_Cortex': 0.005725070121841921, 'Artery...",44,44,12,12
ENSG00000001084,GCLC,"(Brain_Cortex, Artery_Tibial, Cells_Cultured_f...",33,"[21.74736589643256, 4.70064578668365, 2.249949...","{'Brain_Cortex': 0.024826936262456585, 'Artery...",47,47,24,24
ENSG00000001167,NFYA,"(Brain_Cortex, Artery_Tibial, Cells_Cultured_f...",40,[38.28994327673914],"{'Brain_Cortex': 0.016199723974989217, 'Artery...",48,48,14,14


### Save

In [141]:
# this is important, other scripts depend on gene_name to be unique
assert spredixcan_genes_models["gene_name"].is_unique

In [142]:
assert not spredixcan_genes_models.isna().any(None)

In [143]:
spredixcan_genes_models.to_pickle(OUTPUT_DIR_BASE / "gene_tissues.pkl")